In [1]:
## interactive plot
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
## std plots
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")    

In [3]:
## model, data
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import math, time
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import joblib
from torch.utils.data import DataLoader, random_split
from sklearn.preprocessing import MinMaxScaler 
from datetime import timedelta
import mlflow
from itertools import permutations
mlflow.set_tracking_uri("https://mlflow.internal.gaivota.ai/")
from sklearn.metrics import mean_squared_error

In [4]:
from lstm.lstm_model import LSTM, BILSTM
from data.dataset import PriceDataset, price_to_per
from util.metrics import *
from util.learner import Trainer, Tester

In [5]:
from util.losses import MCFD

## Config

In [6]:
# save config for mlflow
config = {
    'lookback':60,
    'lookforward':20,
    'input_dim':4,
    'hidden_dim':32,
    'num_layers':2,
    'output_dim':4,
    'num_epochs':25,
    'learn_rate':0.0001,
    'name':'lstm_v0',
    'batch_size': 60,
    'bidirectional': False,
    'train_dataset':'soybean_price_1990to2019.csv',
    'test_dataset':'soybean_price_2020.csv',
    'loss': 'MSE'
}

DATASET_PATH = "../data/soybean_price_1990to2019.csv"
DATASET_TEST_PATH = "../data/soybean_price_2020.csv"
MODEL_PATH = '../models/'
PIPELINE_PATH = '../pipelines/'

In [7]:
hidden_layers = [8,16,32]
bidirectional = [True,False]
base = [5, 10, 20, 30, 40 , 50 , 60]
hyper_param = []
for per in permutations(base, 2):
    if per[0] >= per[1]:
        hyper_param.append([per[0], per[1]])
for item in base:
    hyper_param.append([item, item])
# ARIMA SETTING
hyper_param.append([4,2])

# Sorting
def sqdist(vector):
    return sum(x*x for x in vector)
hyper_param.sort(key=sqdist)

## Load bases

In [8]:
df_prices = pd.read_csv(DATASET_PATH)
df_prices['Date'] = pd.to_datetime(df_prices['Date'] , format='%Y-%m-%d' )

In [9]:
df_prices_test = pd.read_csv(DATASET_TEST_PATH)
df_prices_test['Date'] = pd.to_datetime(df_prices_test['Date'] , format='%Y-%m-%d' )
df_prices_test['week'] = df_prices_test['Date'].dt.weekday

## Function Train

In [10]:
allowed_dates = df_prices_test['Date'].dt.date.values.tolist()
weekends_dates = df_prices_test.query('week >= 5')['Date'].dt.date.values.tolist()

In [11]:
def date_by_adding_business_days(from_date, add_days,holidays, allowed_dates, weekends_dates):
    business_days_to_add = add_days
    current_date = from_date
    while business_days_to_add > 0:
        current_date += timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5: # sunday = 6
            if not (current_date in weekends_dates):
                continue
        elif current_date in holidays:
            continue
        elif not (current_date in allowed_dates):
            continue
        business_days_to_add -= 1
    return current_date

In [12]:
def preprare_val_data(model, scaler, val_data, allowed_dates,config, weekends_dates):
    res = pd.DataFrame()
    
    for item in val_data:
        with torch.no_grad():
            temp = model(item['x']).detach().numpy()
            for n in range(len(temp)):
                res_temp = pd.DataFrame(scaler.inverse_transform(temp[n]), columns = ['Open','High','Low','Close'])
                start_date = [date_by_adding_business_days(pd.to_datetime(item['date'][n], format =  '%Y-%m-%d'), config['lookback'], [], allowed_dates, weekends_dates)]
                for c in range(1,config['lookforward']):
                    start_date.append(date_by_adding_business_days(start_date[0], c, [], allowed_dates, weekends_dates))
                res_temp['Date'] = start_date
                res = pd.concat([res,res_temp ])
    return res

In [13]:
def train(df_dataset, config, prop = 0.2):
    # Init Model
    if config['bidirectional']:
        model = BILSTM(input_dim = config['input_dim'], 
                       hidden_dim = config['hidden_dim'], 
                       output_dim = config['output_dim'], 
                       num_layers = config['num_layers'], 
                       lookforward = config['lookforward'],
                       bidirectional = config['bidirectional'])
    else:
        model = LSTM(input_dim = config['input_dim'], 
                     hidden_dim = config['hidden_dim'], 
                     output_dim = config['output_dim'], 
                     num_layers = config['num_layers'], 
                     lookforward = config['lookforward'],
                     bidirectional = config['bidirectional'])
    #criterion = MCFD()
    criterion = torch.nn.MSELoss(reduction = 'mean')
    optimizer = torch.optim.Adam(model.parameters(), lr = config['learn_rate'])
    
    # Prepare Dataset
    scaler = MinMaxScaler((-1,1))
    df_dataset_temp = pd.DataFrame(scaler.fit_transform(df_dataset[['Open', 'Close', 'High', 'Low']].values), columns = ['Open', 'Close', 'High', 'Low'])
    df_dataset_temp['Date'] = df_dataset['Date']
    
    allowed_dates = df_dataset_temp['Date'].dt.date.values.tolist()
    dataset = PriceDataset(df_dataset_temp, lookback = config['lookback'], lookforward = config['lookforward'])
    n_val = int(len(dataset) * prop)
    n_train = len(dataset) - n_val
    train_ds, val_ds = random_split(dataset, [n_train, n_val])

    trainer_loader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=False,  num_workers = 1)
    valider_loader = DataLoader(val_ds,   batch_size=config['batch_size'], shuffle=False,  num_workers = 1)

    val_parts = int(len(val_ds)/config['batch_size'])
    # train
    hist_train = np.zeros(config['num_epochs'])
    hist_val = np.zeros(config['num_epochs'])
    start_time = time.time()

    # Init Learner
    train_config = {
        'epochs': config['num_epochs'],
        'batch_size': config['batch_size'],
        'learning_rate': config['learn_rate'],
        'optimizer': optimizer,
        'criterion': criterion,
        'data_loader': trainer_loader}
    
    val_config = {
        'epochs': config['num_epochs'],
        'batch_size': config['batch_size'],
        'learning_rate': config['learn_rate'],
        'optimizer': optimizer,
        'criterion': criterion,
        'data_loader': valider_loader}


    trainer = Trainer(model, train_config)
    validater = Tester(model, val_config)

    for t in tqdm(range(config['num_epochs'])):
        hist_train[t] = trainer.run_epoch()
        hist_val[t] = validater.run_epoch()

    training_time = time.time()-start_time
    print("")
    print("MSE: ", hist_val[-1])
    print("Training time: {}".format(training_time))
   
    return model, scaler, hist_train, hist_val, valider_loader

## Multi trainer

In [14]:
result = pd.DataFrame()
mlflow_active = True

In [15]:
for hypeparam in hyper_param:
    for hidden in hidden_layers:
        for bi in bidirectional:
            config['lookback'] = hypeparam[0]
            config['lookforward'] = hypeparam[1]
            config['hidden_dim'] = hidden
            config['bidirectional'] = bi
            if config['bidirectional']:
                config['num_layers'] = 1
            base_name = "lstm_{}_for_{}_{}_hidden_layers_bi_{}".format(hypeparam[0], hypeparam[1], hidden, bi)
            config['name'] = base_name
            # Train model
            model, scaler, hist_train, hist_val, training_time = train(df_prices, config, prop = 0.2)
            
            # Error in TEST DATASET
            df_test_scaler = pd.DataFrame(scaler.transform(df_prices_test[['Open', 'Close', 'High', 'Low']].values), columns = ['Open', 'Close', 'High', 'Low'])
            df_test_scaler['Date'] = df_prices_test['Date']
            dataset = PriceDataset(df_test_scaler,  lookback = config['lookback'], lookforward = config['lookforward'], input_dim = config['input_dim'])
            test_data = DataLoader(dataset, batch_size = config['batch_size'], shuffle=False,  num_workers = 1)
            res = preprare_val_data(model, scaler, test_data, allowed_dates, config, weekends_dates)
            res = res.sort_values("Date", ascending = True)
            res = res.drop_duplicates(subset = ["Date"])
            res = res.reset_index(drop=True)
            comp = df_prices_test[df_prices_test['Date'].isin(res['Date'].values)]
            comp = comp.sort_values("Date", ascending = True)
            comp = comp.reset_index(drop=True)
            
            print('metrics - mlflow')
            row = {'lookback':config['lookback'],
                   'lookforward':config['lookforward'],
                   'hidden_dim':config['hidden_dim'],
                   'bidirectional':config['bidirectional'],
                   'TEST_MAPE':value_mape(comp, res, ['Open', 'High', 'Low', 'Close']),
                   'TEST_PER_CHANGE_MAPE':per_change_mape(res, comp, ['Open', 'High', 'Low', 'Close']),
                   'ACC_DAILY_TEND_DIR':acc_daily_trend_direction(comp, res, ['Open', 'High', 'Low', 'Close']),
                   'ACC_WEEKLY_TEND_DIR':acc_weekly_trend_direction(comp, res, ['Open', 'High', 'Low', 'Close']),
                   'ACC_MONTHLY_TEND_DIR':acc_monthly_trend_direction(comp, res, ['Open', 'High', 'Low', 'Close']),
                   'MEAN_FORECAST_ERROR':mean_forecast_error(comp,res, ['Open', 'High', 'Low', 'Close'])}
            result = result.append(row, ignore_index = True)
            
            if mlflow_active:
                mlflow.set_experiment('gaivota-python-market-sentiment-analysis-lstm')
                mlflow.start_run()
                mlflow.log_params(config)
                for t in range(config['num_epochs']):            
                    mlflow.log_metrics({'TRAIN_LOSS':hist_train[t]}, step = t)
                    mlflow.log_metrics({'VAL_LOSS':hist_val[t]}, step = t)
                for col in  ['Open', 'High', 'Low', 'Close']:
                    mlflow.log_metric('TEST_MAPE_{}'.format(col), row['TEST_MAPE'][col])
                    mlflow.log_metric('TEST_PER_CHANGE_MAPE_{}'.format(col), row['TEST_PER_CHANGE_MAPE'][col])
                    mlflow.log_metric('TEST_ACC_DAILY_TEND_DIR_{}'.format(col), row['ACC_DAILY_TEND_DIR'][col])
                    mlflow.log_metric('TEST_ACC_WEEKLY_TEND_DIR_{}'.format(col), row['ACC_WEEKLY_TEND_DIR'][col])
                    mlflow.log_metric('TEST_ACC_MONTHLY_TEND_DIR_{}'.format(col), row['ACC_MONTHLY_TEND_DIR'][col])
                    mlflow.log_metric('TEST_MEAN_FORECAST_ERROR_{}'.format(col), row['MEAN_FORECAST_ERROR'][col])
                mlflow.end_run()
            print("{}: {} - {}".format(base_name, row['TEST_PER_CHANGE_MAPE'], row['ACC_DAILY_TEND_DIR']),)
            print("--------------------------------------------------------------------------------")

100%|██████████| 25/25 [00:08<00:00,  2.88it/s]


MSE:  0.0054918884550436186
Training time: 8.696808338165283


metrics - mlflow
lstm_4_for_2_8_hidden_layers_bi_True: {'Open': 12.14, 'High': 4.79, 'Low': 5.16, 'Close': 5.39} - {'Open': 56.52, 'High': 50.99, 'Low': 50.99, 'Close': 49.01}
--------------------------------------------------------------------------------


100%|██████████| 25/25 [00:06<00:00,  3.58it/s]


MSE:  0.012270709302706214
Training time: 6.993438720703125


...


MSE:  0.013479024225559372
Training time: 89.36898350715637


metrics - mlflow
lstm_60_for_60_32_hidden_layers_bi_True: {'Open': 2.38, 'High': 1.71, 'Low': 1.69, 'Close': 9.91} - {'Open': 50.25, 'High': 48.73, 'Low': 56.35, 'Close': 56.35}
--------------------------------------------------------------------------------


100%|██████████| 25/25 [00:47<00:00,  1.92s/it]


MSE:  0.013559250841633631
Training time: 47.91288685798645


metrics - mlflow
lstm_60_for_60_32_hidden_layers_bi_False: {'Open': 1.62, 'High': 1.48, 'Low': 1.98, 'Close': 1.46} - {'Open': 46.19, 'High': 55.33, 'Low': 51.78, 'Close': 52.28}
--------------------------------------------------------------------------------


----------------------------